来源：[Fuzzing: Breaking Things with Random Inputs](https://www.fuzzingbook.org/html/Fuzzer.html)

## 模糊测试的简单事例

In [ ]:
# 生成一个指定范围内，随机长度，随机字母的字符串
import random
def fuzzer(min_length=5,max_length=15,char_start=ord('a'),char_range=26):
    str_length = random.randrange(min_length,max_length+1)
    out = ""
    for i in range(str_length):
        out += chr(random.randrange(char_start,char_start+char_range))
    return out

In [ ]:
# 将生成的字符串写入文件
import os
import tempfile
basename = "input.txt"
tmp_dir = tempfile.mkdtemp()
FILE = os.path.join(tmp_dir,basename)

data = fuzzer()
with open(FILE,"w") as f:
    f.write(data)
print(open(FILE).read())

In [ ]:
# 调用外部程序
import subprocess
program = "bc"
with open(FILE, "w") as f:
    f.write("2 + 2\n")
result = subprocess.run([program, FILE],
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)  # Will be "text" in Python 3.7
# print(result)
print(result.stdout)
# print(result.returncode)
# print(result.stderr)

In [ ]:
# 测试bc程序
trials = 50
program = "bc"
results = []
for i in range(trials):
    data = fuzzer(min_length=2,char_start=ord('0'))
    data += '\n'
    with open(FILE,"w") as f:
        f.write(data)
    result = subprocess.run([program, FILE],
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)  # Will be "text" in Python 3.7
    results.append((data,result))

In [ ]:
# 测试bc程序的结果分析，没有返回值为非0存在，即没有崩溃
# 如果有返回代码非0，断言
sum_suc = 0
for i in range(trials):
    (data,result) = results[i]
    assert result.returncode == 0
    if result.stderr == "":
        sum_suc += 1
        print(f"suc：{data}-->{result.stdout}")
    else:
        print(f"fail：{data}-->{result.stderr}")
print(sum_suc)

In [ ]:
# 创建可能会内存访问越界的程序
with open("program.c", "w") as f:
    f.write("""
#include <stdlib.h>
#include <string.h>

int main(int argc, char** argv) {
    /* Create an array with 100 bytes, initialized with 42 */
    char *buf = malloc(100);
    memset(buf, 42, 100);

    /* Read the N-th element, with N being the first command-line argument */
    int index = atoi(argv[1]);
    char val = buf[index];

    /* Clean up memory so we don't leak */
    free(buf);
    return val;
}
    """)

In [ ]:
# 使用clang编译；AddressSanitizer is a fast memory error detector.
# gcc应该也有这个功能吧
!gcc -fsanitize=address -g -o program program.c
# !clang -fsanitize=address -g -o program program.c

In [ ]:
# 正常，输出42
!./program 99; echo $?

In [ ]:
# 错误，SUMMARY: AddressSanitizer: heap-buffer-overflow
!./program 100; echo $?

In [ ]:
# 删除程序
!rm -fr program program.*

## 构建模糊测试架构

In [ ]:
# base runner class 
class Runner(object):
    # Test outcomes
    PASS = "PASS"
    FAIL = "FAIL"
    UNRESOLVED = "UNRESOLVED"
    def __init__(self):
        pass
    def run(self,inp):
        return(inp,Runner.UNRESOLVED)